In [1]:
using DelimitedFiles

In [2]:
function read_par_BM_0gap(path::AbstractString, q::Integer = 21)    
    params = readdlm(path,' ', use_mmap = true)[:, 2:6]
    l_file = size(params, 1) 
    N = Integer(((q - 2) + sqrt( (q-2)^2 + 8*l_file))/(2*q))
    J = Array{Float64}(undef, q, q, N, N)
    h = Array{Float64}(undef, q, N)
    n_J = Int(q*q*N*(N-1)/2)
    n_h = q*N

    for k in 1:n_J
        i, j, a, b, par_j = params[k, :]
        i += 1
        j += 1
        a == 0 && (a = 21)
        b == 0 && (b = 21)
        J[a, b, i, j] = par_j
    end

    for l in (n_J + 1): n_h + n_J
        i, a, par_h = params[l, :]
        i += 1
        a == 0 && (a = 21)
        h[a, i] = par_h
    end

    return h, J
end

read_par_BM_0gap (generic function with 2 methods)

In [11]:
# Function to determine L from the .dat file
function determine_L(file_path::String)
    max_i = 0
    max_j = 0
    
    # Open the file and read lines
    open(file_path, "r") do file
        for line in eachline(file)
            if startswith(line, "J")
                # Parse the J line
                _, i_str, j_str, _, _, _ = split(line)
                i = parse(Int, i_str)
                j = parse(Int, j_str)
                
                # Update max_i and max_j
                max_i = max(max_i, i)
                max_j = max(max_j, j)
            elseif startswith(line, "h")
                # Parse the h line
                _, i_str, _ = split(line)
                i = parse(Int, i_str)
                
                # Update max_i
                max_i = max(max_i, i)
            end
        end
    end
    
    # L is the maximum of max_i and max_j + 1 (to account for zero-based index)
    return max(max_i, max_j) + 1
end

# Function to read and parse the .dat file
function read_parameters(file_path::String)
    # Determine L dynamically
    L = determine_L(file_path)
    
    # Initialize arrays with zeros
    h = zeros(Float64, 21, L)
    J = zeros(Float64, 21, L, 21, L)
    
    # Open the file and read lines
    open(file_path, "r") do file
        for line in eachline(file)
            if startswith(line, "J")
                # Parse the J line
                _, i_str, j_str, a_str, b_str, value_str = split(line)
                i = parse(Int, i_str) + 1
                j = parse(Int, j_str) + 1
                a = parse(Int, a_str) + 1  # Convert to 1-based index
                b = parse(Int, b_str) + 1  # Convert to 1-based index
                value = parse(Float64, value_str)
                
                # Assign value to J only if i and j are different
                if i != j
                    J[a, i, b, j] = value
                    J[b, j, a, i] = value
                end
            elseif startswith(line, "h")
                # Parse the h line
                _, i_str, a_str, value_str = split(line)
                i = parse(Int, i_str) + 1
                a = parse(Int, a_str) + 1  # Convert to 1-based index
                value = parse(Float64, value_str)
                
                # Assign value to h
                h[a, i] = value
            end
        end
    end
    
    return h, permutedims(J, [1,3,2,4])
end

# Specify the file path (you need to replace "path_to_file.dat" with the actual file path)
file_path = "../../Parameters_conv_DHFR.dat"

# Read the parameters
h, J = read_parameters(file_path);

println(sum(abs2, h))
println(sum(abs2, J))



21109.06876327675
4534.848277649538


In [12]:
using JLD2

In [13]:
@save "../../new_pars_DHFR.jld2" h J

In [6]:
PATH_PAR = "../../Parameters_conv_DHFR.dat"
h_new, J_new = read_par_BM_0gap(PATH_PAR);

In [7]:
println(sum(abs2, h_new))
println(sum(abs2, J_new))

21109.06876327675
2267.4241388247688


In [8]:
println(sum(h_new .^2))
println(sum(J_new .^2))

21109.06876327675
2267.424138824769


In [10]:
println(sum(abs2, J_new - permutedims(J_new, [2,1,4,3]))) 

4534.848277649538


In [10]:
L = size(J,3)
for i in 1:L
    for j in 1:L
        @assert J[:,i,:,j] == J[:,j,:,i]' "$i, $j"
    end
end

In [18]:
println(J_new[:,1,1,2] -J_new[1,:,2,1]))

[1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 0.0, 1.3e-5, -3.4e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.3e-5, 1.0e-5, 1.3e-5, 1.3e-5, 1.3e-5, -0.00023]


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/disat/ldibari/Gen.jl/notebooks/In[18]#1:40\[90mIn[18]:1:40[0;0m]8;;\
println(J_new[:,1,1,2] -J_new[1,:,2,1])[48;2;120;70;70m)[0;0m
[90m#                                      ╙ ── [0;0m[91mextra tokens after end of expression[0;0m

In [17]:
@assert J_new[:,1,1,2] == J_new[1,:,2,1]


LoadError: AssertionError: J_new[:, 1, 1, 2] == J_new[1, :, 2, 1]

In [19]:
J_new[1,:,2,1]

21-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0